In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
adult_names  = pd.read_table('adult_names.txt').iloc[91:]
adult_names.columns = ['name']
column_name = adult_names['name'].str.split(':', expand= True)[0]
columns =  column_name.iloc[1:] 
columns = columns.append(column_name.iloc[0:1], ignore_index=True)
columns

0                age
1          workclass
2             fnlwgt
3          education
4      education-num
5     marital-status
6         occupation
7       relationship
8               race
9                sex
10      capital-gain
11      capital-loss
12    hours-per-week
13    native-country
14      >50K, <=50K.
Name: 0, dtype: object

In [3]:
adult = pd.read_csv('adult_data.csv', header= None)
adult.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
adult.columns = [col for col in columns]
adult.head()
#adult.describe()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,">50K, <=50K."
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


### 1. Create an sqlalchemy engine using a sample from the data set

In [5]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///sqldb.db')
#engine.execute('DROP table adult')
adult.to_sql('adult', con=engine, if_exists='replace', index_label='id')

In [6]:
rows = engine.execute("SELECT sql FROM sqlite_master WHERE name like '%adult%' ")
for row in rows:
    print(row, '\n')

('CREATE TABLE "adult.head()" (\n\t"index" BIGINT, \n\tage BIGINT, \n\tworkclass TEXT, \n\tfnlwgt BIGINT, \n\teducation TEXT, \n\t"education-num" BIGIN ... (90 characters truncated) ... n\tsex TEXT, \n\t"capital-gain" BIGINT, \n\t"capital-loss" BIGINT, \n\t"hours-per-week" BIGINT, \n\t"native-country" TEXT, \n\t">50K, <=50K." TEXT\n)',) 

('CREATE INDEX "ix_adult.head()_index" ON "adult.head()" ("index")',) 

('CREATE TABLE adult (\n\tid BIGINT, \n\tage BIGINT, \n\tworkclass TEXT, \n\tfnlwgt BIGINT, \n\teducation TEXT, \n\t"education-num" BIGINT, \n\t"marita ... (76 characters truncated) ... n\tsex TEXT, \n\t"capital-gain" BIGINT, \n\t"capital-loss" BIGINT, \n\t"hours-per-week" BIGINT, \n\t"native-country" TEXT, \n\t">50K, <=50K." TEXT\n)',) 

('CREATE INDEX ix_adult_id ON adult (id)',) 



### write select queries

In [7]:
rows = engine.execute('''select * from adult limit 5''')

for row in rows:
    print(row, '\n')

(0, 39, ' State-gov', 77516, ' Bachelors', 13, ' Never-married', ' Adm-clerical', ' Not-in-family', ' White', ' Male', 2174, 0, 40, ' United-States', ' <=50K') 

(1, 50, ' Self-emp-not-inc', 83311, ' Bachelors', 13, ' Married-civ-spouse', ' Exec-managerial', ' Husband', ' White', ' Male', 0, 0, 13, ' United-States', ' <=50K') 

(2, 38, ' Private', 215646, ' HS-grad', 9, ' Divorced', ' Handlers-cleaners', ' Not-in-family', ' White', ' Male', 0, 0, 40, ' United-States', ' <=50K') 

(3, 53, ' Private', 234721, ' 11th', 7, ' Married-civ-spouse', ' Handlers-cleaners', ' Husband', ' Black', ' Male', 0, 0, 40, ' United-States', ' <=50K') 

(4, 28, ' Private', 338409, ' Bachelors', 13, ' Married-civ-spouse', ' Prof-specialty', ' Wife', ' Black', ' Female', 0, 0, 40, ' Cuba', ' <=50K') 



In [8]:
rows = engine.execute('''select age, workclass, "marital-status" from adult limit 5''')

for row in rows:
    print(row, '\n')

(39, ' State-gov', ' Never-married') 

(50, ' Self-emp-not-inc', ' Married-civ-spouse') 

(38, ' Private', ' Divorced') 

(53, ' Private', ' Married-civ-spouse') 

(28, ' Private', ' Married-civ-spouse') 



### 2. Write two basic update queries

In [9]:
engine.execute('''update adult set "native-country" ='USA' where "native-country"=' United-States' ''')

In [10]:
engine.execute('''update adult set workclass ='Private' where workclass=' Private' ''')

### 3. Write two delete queries

In [11]:
engine.execute('''delete from adult where id=0 ''')

In [12]:
engine.execute('''delete from adult where "hours-per-week"<20 ''')

### 4. Write two filter queries

In [13]:
rows = engine.execute('''select age, workclass, "marital-status" from adult where id>5 limit 5''')

for row in rows:
    print(row, '\n')

(52, ' Self-emp-not-inc', ' Married-civ-spouse') 

(31, 'Private', ' Never-married') 

(42, 'Private', ' Married-civ-spouse') 

(37, 'Private', ' Married-civ-spouse') 

(30, ' State-gov', ' Married-civ-spouse') 



In [14]:
rows = engine.execute('''select age, workclass, "marital-status" from adult where age>50 and "marital-status"=' Married-civ-spouse' limit 5''')

for row in rows:
    print(row, '\n')

(53, 'Private', ' Married-civ-spouse') 

(52, ' Self-emp-not-inc', ' Married-civ-spouse') 

(56, ' Local-gov', ' Married-civ-spouse') 

(54, ' ?', ' Married-civ-spouse') 

(53, ' Self-emp-not-inc', ' Married-civ-spouse') 



### 5. Write two function queries

In [15]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
Session.configure(bind=engine) 
session = Session()
adult.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000


In [16]:
from sqlalchemy import func
from sqlalchemy import text

In [30]:
from sqlalchemy import (Table, Column, String, Integer, MetaData, select, func)

metadata = MetaData()

adult = Table('adult', metadata,
              Column('age', Integer),
              Column('workclass', String),
              Column('education', String),
              Column('occupation', String),
             
)

In [18]:
for t in metadata.sorted_tables:
    print(t.name)

adult


In [24]:
#adult.columns.age
#session.query(func.count(adult.columns.age)).all()
session.query(func.count(adult.columns.workclass)).all()

[(30856)]

In [20]:
session.query(adult.columns.age, func.count(adult.columns.age)).group_by(adult.columns.age).all()

[(17, 228),
 (18, 413),
 (19, 601),
 (20, 657),
 (21, 641),
 (22, 697),
 (23, 826),
 (24, 768),
 (25, 817),
 (26, 761),
 (27, 820),
 (28, 844),
 (29, 798),
 (30, 839),
 (31, 874),
 (32, 810),
 (33, 862),
 (34, 864),
 (35, 863),
 (36, 882),
 (37, 844),
 (38, 814),
 (39, 799),
 (40, 781),
 (41, 790),
 (42, 767),
 (43, 759),
 (44, 709),
 (45, 716),
 (46, 727),
 (47, 694),
 (48, 531),
 (49, 569),
 (50, 587),
 (51, 585),
 (52, 469),
 (53, 461),
 (54, 407),
 (55, 407),
 (56, 358),
 (57, 344),
 (58, 351),
 (59, 343),
 (60, 297),
 (61, 279),
 (62, 237),
 (63, 205),
 (64, 171),
 (65, 156),
 (66, 125),
 (67, 113),
 (68, 97),
 (69, 80),
 (70, 68),
 (71, 46),
 (72, 45),
 (73, 41),
 (74, 29),
 (75, 29),
 (76, 28),
 (77, 17),
 (78, 14),
 (79, 17),
 (80, 16),
 (81, 11),
 (82, 6),
 (83, 5),
 (84, 5),
 (85, 2),
 (86, 1),
 (88, 3),
 (90, 36)]

In [31]:
session.query(adult).filter(adult.c.age == 39).all()

[(39, 'Private', ' HS-grad', ' Exec-managerial'),
 (39, 'Private', ' Some-college', ' Craft-repair'),
 (39, ' Federal-gov', ' Assoc-acdm', ' Exec-managerial'),
 (39, ' ?', ' Masters', ' ?'),
 (39, ' Self-emp-not-inc', ' HS-grad', ' Exec-managerial'),
 (39, 'Private', ' HS-grad', ' Sales'),
 (39, 'Private', ' HS-grad', ' Other-service'),
 (39, ' Local-gov', ' 12th', ' Tech-support'),
 (39, 'Private', ' HS-grad', ' Handlers-cleaners'),
 (39, 'Private', ' HS-grad', ' Handlers-cleaners'),
 (39, 'Private', ' HS-grad', ' Machine-op-inspct'),
 (39, 'Private', ' Bachelors', ' Craft-repair'),
 (39, 'Private', ' Some-college', ' Farming-fishing'),
 (39, 'Private', ' HS-grad', ' Transport-moving'),
 (39, 'Private', ' Bachelors', ' Exec-managerial'),
 (39, ' Self-emp-inc', ' Bachelors', ' Exec-managerial'),
 (39, ' Self-emp-inc', ' HS-grad', ' Exec-managerial'),
 (39, ' Self-emp-inc', ' 9th', ' Exec-managerial'),
 (39, 'Private', ' HS-grad', ' Transport-moving'),
 (39, 'Private', ' HS-grad', ' Tra